<a href="https://colab.research.google.com/github/raghnyaa/python-code/blob/master/ClinicalTrials_NanoCTData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import psycopg2
from psycopg2.extras import DictCursor
import re
import pandas as pd

# Connect to your PostgreSQL database
conn = psycopg2.connect(
host = "aact-db.ctti-clinicaltrials.org",
port = "5432",
dbname = "aact",
user = "rvalluru",
password = "Prakhya@09!",
)

# Define the keywords to search for
keywords = [
    'nano', 'liposom', 'abelcet', 'abraxane', 'daunoxome', 'depocyt', 'doxil',
    'eligard', 'feraheme', 'feridex', 'ferumoxsil', 'ferumoxtran-10',
    'genexol', 'kadcyla', 'lipoplatin', 'marqibo', 'mepact', 'myocet', 'ontak',
    'pluronic f127', 'pluronic p105', 'rapamune', 'thermodox', 'tubastatin a',
    'visudyne', 'dendrimer', 'micell'
]


# Create a cursor with a dictionary cursor
with conn.cursor(cursor_factory=DictCursor) as cur:
    # Execute the SQL query to get the combined text
    cur.execute("""
    SELECT DISTINCT
        STUDIES.NCT_ID as NCT_ID,
        COALESCE(DETAILED_DESCRIPTIONS.DESCRIPTION, '') || ' ' ||
        COALESCE(BRIEF_SUMMARIES.DESCRIPTION, '') || ' ' ||
        COALESCE(STUDIES.BRIEF_TITLE, '') AS combined_text
    FROM STUDIES
    LEFT JOIN BRIEF_SUMMARIES ON STUDIES.NCT_ID = BRIEF_SUMMARIES.NCT_ID
    LEFT JOIN INTERVENTIONS ON STUDIES.NCT_ID = INTERVENTIONS.NCT_ID
    LEFT JOIN CONDITIONS ON STUDIES.NCT_ID = CONDITIONS.NCT_ID
    LEFT JOIN DETAILED_DESCRIPTIONS ON DETAILED_DESCRIPTIONS.NCT_ID = STUDIES.NCT_ID
    """)

    # Fetch all rows
    results = cur.fetchall()
    print(f"Number of rows returned: {len(results)}")

    # Process each row to find matching keywords
    nct_keyword_map = {}
    for row in results:
        nct_id = row['nct_id']
        combined_text = row['combined_text'].lower()

        # Find matched "nano" related words and other keywords using regex
        matched_nano_words = set(re.findall(r'\b\w*nano\w*\b', combined_text))
        matched_other_keywords = set(kw for kw in keywords if kw in combined_text)

        # Filter out duplicates and empty matches
        matched_nano_words = {word for word in matched_nano_words if word}
        matched_other_keywords = {word for word in matched_other_keywords if word}

        if matched_nano_words or matched_other_keywords:
            nct_keyword_map[nct_id] = {
                'nano_keywords': list(matched_nano_words),
                'other_keywords': list(matched_other_keywords)
            }


 # Convert the dictionary to a DataFrame
    df = pd.DataFrame.from_dict(nct_keyword_map, orient='index')

    # Save the DataFrame to Google Drive
    output_path = '/content/drive/My Drive/nano_keywords_results.xlsx'
    df.to_excel(output_path, index=True)

    print(f"Results saved to: {output_path}")

# Close the connection
conn.close()

Number of rows returned: 509736
Results saved to: /content/drive/My Drive/nano_keywords_results.xlsx
